In [1]:
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append('..')
from src.util import load_c4_dataset
from src.datasets import C4Dataset
from transformers import AutoTokenizer

c:\Users\Simon\anaconda3\envs\master_thesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_path = '../dataset/c4/en/c4-train.{:05d}-of-01024.json'

In [3]:
list_of_texts = load_c4_dataset(base_path, 1)


Loading Dataframe with c4 Data...: 100%|██████████| 1/1 [00:08<00:00,  8.94s/it]


In [4]:
model_name = "google/t5-v1_1-base"
print("Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading Tokenizer...


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
c4_dataset = C4Dataset(list_of_texts, tokenizer)

In [6]:
input, target = c4_dataset[1]
print(f"Input Sequence: {input}")
print(f"Target Sequence: {target}")

Length of Total tokens: 440
66
 Span Lengths: [5 4 4 2 2 3 5 1 2 5 8 1 1 5 1 1 2 2 2 3 3 4]
Span Starts: [191, 279, 299, 306, 311, 338, 346, 356, 357, 362, 367, 386, 391, 394, 403, 405, 406, 409, 412, 417, 421, 427]
Length of Target IDs: 88
Input Sequence: Discussion in 'Mac OS X Lion (10.7)' started by axboi87, Jan 20, 2012. I've got a 500gb internal drive and a 240gb SSD. When trying to restore using disk utility i'm given the error "Not enough space on disk ____ to restore" But I shouldn't have to do that!!! Any ideas or workarounds before resorting to the above? Use Carbon Copy Cloner to copy one drive to the other. I've done this several times going from larger HDD to smaller SSD and I wound up with a bootable SSD drive. One step you have to remember not to skip is to use Disk Utility to partition the SSD as GUID partition scheme HFS+ before doing the clone. If it came Apple Partition Scheme, even if you let CCC do the clo<extra_id_0>resulting drive won't be bootable. CCC usually 